### EDA: Raw Flight Data

**Objective:** Explore the schema and data quality of flight data in DuckDB.

**Data source:** `data_warehouse/svenska-flyt.duckdb`  

**Tables:** `flights_arrivals_raw`, `flights_departures_raw`

#### 1. Setup & Connection

In [1]:
# Load SQL magic extension for DuckDB
%load_ext sql

Tip: You may define configurations in C:\Users\Katrin\Documents\github\svensk-flyt\pyproject.toml or C:\Users\Katrin\.jupysql\config.

Did not find user configurations in C:\Users\Katrin\Documents\github\svensk-flyt\pyproject.toml.

In [2]:
%config SqlMagic.displaylimit = 100

In [3]:
# Connect to DuckDB database
%sql duckdb:///../data_warehouse/svenska-flyt.duckdb

Connecting to 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

## 2. Database Overview

In [4]:
%%sql
-- List all tables in the flights schema
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'flights';

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

table_name
flights_arrivals_raw
flights_arrivals_raw__code_share_data
flights_arrivals_raw__remarks_english
flights_arrivals_raw__remarks_swedish
flights_arrivals_raw__via_destinations
flights_departures_raw
flights_departures_raw__code_share_data
flights_departures_raw__remarks_english
flights_departures_raw__remarks_swedish
flights_departures_raw__via_destinations


In [5]:
%%sql
-- Row counts for each table
SELECT 
    'flights_arrivals_raw' AS table_name,
    COUNT(*) AS row_count
FROM flights.flights_arrivals_raw

UNION ALL

SELECT 
    'flights_departures_raw' AS table_name,
    COUNT(*) AS row_count
FROM flights.flights_departures_raw;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

table_name,row_count
flights_arrivals_raw,1801
flights_departures_raw,1801


## 3. Schema Exploration - Arrivals

In [6]:
%%sql
-- Get all column names and types for arrivals
DESCRIBE flights.flights_arrivals_raw;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

column_name,column_type,null,key,default,extra


In [7]:
%%sql
-- Sample arrivals data (first 5 rows)
SELECT * 
FROM flights.flights_arrivals_raw 
LIMIT 5;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

flight_id,departure_airport_swedish,departure_airport_english,airline_operator__iata,airline_operator__icao,airline_operator__name,arrival_time__scheduled_utc,location_and_status__terminal,location_and_status__flight_leg_status,location_and_status__flight_leg_status_swedish,location_and_status__flight_leg_status_english,flight_leg_identifier__callsign,flight_leg_identifier__flight_id,flight_leg_identifier__flight_departure_date_utc,flight_leg_identifier__departure_airport_iata,flight_leg_identifier__arrival_airport_iata,flight_leg_identifier__departure_airport_icao,flight_leg_identifier__arrival_airport_icao,di_indicator,_dlt_load_id,_dlt_id,baggage__baggage_claim_unit,arrival_time__estimated_utc,arrival_time__actual_utc,location_and_status__gate,baggage__first_bag_utc,baggage__last_bag_utc,flight_leg_identifier__aircraft_registration,flight_leg_identifier__ssr_code,baggage__estimated_first_bag_utc
FI312,Reykjavik,Reykjavik,FI,ICE,Icelandair,2026-01-25 20:35:00+01:00,T5,DEL,Borttagen,Deleted,ICE40E,FI312,2026-01-25 01:00:00+01:00,KEF,ARN,BIKF,ESSA,S,1769364521.5664544,xbkh6ZJBXNr9nA,None,None,None,None,None,None,None,None,None
D83194,Köpenhamn,Copenhagen,D8,NSZ,Norwegian,2026-01-25 08:25:00+01:00,T5,DEL,Borttagen,Deleted,NSZ3194,D83194,2026-01-25 01:00:00+01:00,CPH,ARN,EKCH,ESSA,S,1769364521.5664544,IujDqlmG18/klg,None,None,None,None,None,None,None,None,None
D82610,Helsingfors,Helsinki,D8,NSZ,Norwegian,2026-01-25 10:00:00+01:00,T5,DEL,Borttagen,Deleted,NSZ2610,D82610,2026-01-25 01:00:00+01:00,HEL,ARN,EFHK,ESSA,S,1769364521.5664544,9kVVIb9buion2w,None,None,None,None,None,None,None,None,None
TP780,Lissabon,Lisbon,TP,TAP,TAP Portugal,2026-01-25 13:30:00+01:00,T5,DEL,Borttagen,Deleted,TAP780,TP780,2026-01-25 01:00:00+01:00,LIS,ARN,LPPT,ESSA,S,1769364521.5664544,tz/jPJpzkadSjQ,None,None,None,None,None,None,None,None,None
EK155,Dubai,Dubai,EK,UAE,Emirates,2026-01-25 19:15:00+01:00,T5,DEL,Borttagen,Deleted,UAE155,EK155,2026-01-25 01:00:00+01:00,DXB,ARN,OMDB,ESSA,I,1769364521.5664544,JG4hCh+x9NbR6w,None,None,None,None,None,None,None,None,None


Note:
According do documentation: 
The status of a FlightLeg.
Possible values are:
SCH Scheduled
FPL Flight Plan
FLS Flight Suspended
SEQ Sequenced
ACT Active
CAN Cancelled
LAN Landed
RER Rerouted
DIV Diverted
DEL Deleted

DEL (Deleted) means these flights have been removed from the schedule. This explains why:

All the sample flights show location_and_status__flight_leg_status = DEL
All actual times are NULL
Gate and baggage info are NULL
This is important for data quality — there will be a need to filter out deleted flights in the analysis (or track them separately for cancellation metrics).

## 4. Schema Exploration - Departures

In [8]:
%%sql
-- Get all column names and types for departures
DESCRIBE flights.flights_departures_raw;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

column_name,column_type,null,key,default,extra


In [9]:
%%sql
-- Sample departures data (first 5 rows)
SELECT * 
FROM flights.flights_departures_raw 
LIMIT 5;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

flight_id,arrival_airport_swedish,arrival_airport_english,airline_operator__iata,airline_operator__icao,airline_operator__name,departure_time__scheduled_utc,location_and_status__terminal,location_and_status__flight_leg_status,location_and_status__flight_leg_status_swedish,location_and_status__flight_leg_status_english,flight_leg_identifier__callsign,flight_leg_identifier__flight_id,flight_leg_identifier__flight_departure_date_utc,flight_leg_identifier__departure_airport_iata,flight_leg_identifier__arrival_airport_iata,flight_leg_identifier__departure_airport_icao,flight_leg_identifier__arrival_airport_icao,di_indicator,_dlt_load_id,_dlt_id,check_in__check_in_desk_from,check_in__check_in_desk_to,departure_time__actual_utc,location_and_status__gate,location_and_status__gate_action,location_and_status__gate_action_swedish,location_and_status__gate_action_english,location_and_status__gate_open_utc,location_and_status__gate_close_utc,check_in__check_in_status,check_in__check_in_status_swedish,check_in__check_in_status_english,flight_leg_identifier__aircraft_registration,flight_leg_identifier__ssr_code,departure_time__estimated_utc
FI313,Reykjavik,Reykjavik,FI,ICE,Icelandair,2026-01-25 21:30:00+01:00,T5,DEL,Borttagen,Deleted,ICE313,FI313,2026-01-25 01:00:00+01:00,ARN,KEF,ESSA,BIKF,S,1769364521.5664544,MzLSBC6fdp0oDA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
D83195,Köpenhamn,Copenhagen,D8,NSZ,Norwegian,2026-01-25 08:55:00+01:00,T5,DEL,Borttagen,Deleted,NSZ3195,D83195,2026-01-25 01:00:00+01:00,ARN,CPH,ESSA,EKCH,S,1769364521.5664544,Kh6JpaQHI3ZlSg,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
D82611,Helsingfors,Helsinki,D8,NSZ,Norwegian,2026-01-25 10:35:00+01:00,T5,DEL,Borttagen,Deleted,NSZ2611,D82611,2026-01-25 01:00:00+01:00,ARN,HEL,ESSA,EFHK,S,1769364521.5664544,+WvfxlY96qEHog,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
TP781,Lissabon,Lisbon,TP,TAP,TAP Portugal,2026-01-25 14:20:00+01:00,T5,DEL,Borttagen,Deleted,TAP781,TP781,2026-01-25 01:00:00+01:00,ARN,LIS,ESSA,LPPT,S,1769364521.5664544,q7g9n9lMdADyrQ,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
LO456,Warszawa,Warsaw,LO,LOT,LOT,2026-01-25 07:45:00+01:00,T5,DEL,Borttagen,Deleted,LOT4JM,LO456,2026-01-25 01:00:00+01:00,ARN,WAW,ESSA,EPWA,S,1769364521.5664544,Qb31exo2GJC9Sg,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


## 5. Key Fields for KPIs

Explore the fields needed for our 6 research questions:
1. Peak hours → timestamps
2. Punctuality → status, scheduled vs actual times
3. Airline performance → airline codes, delays
4. Route popularity → airport pairs
5. Capacity utilization → flight counts per airport
6. Seasonal trends → dates, day of week

### 1. Peak hours → timestamps

In [10]:
%%sql
-- What statuses do we have?
SELECT 
    location_and_status__flight_leg_status AS status,
    COUNT(*) AS count
FROM flights.flights_arrivals_raw
GROUP BY location_and_status__flight_leg_status
ORDER BY count DESC;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

status,count
LAN,1221
DEL,379
SCH,199
CAN,2


In [11]:
%%sql
-- Peak hours: Group arrivals by scheduled hour (excluding DEL)
SELECT 
    DATE_TRUNC('hour', arrival_time__scheduled_utc) AS arrival_hour,
    COUNT(*) AS flight_count,
    location_and_status__flight_leg_status AS status
FROM flights.flights_arrivals_raw
WHERE location_and_status__flight_leg_status != 'DEL'
GROUP BY DATE_TRUNC('hour', arrival_time__scheduled_utc), location_and_status__flight_leg_status
ORDER BY arrival_hour, flight_count DESC;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

arrival_hour,flight_count,status
2026-01-23 00:00:00+01:00,5,LAN
2026-01-23 01:00:00+01:00,1,LAN
2026-01-23 05:00:00+01:00,1,LAN
2026-01-23 06:00:00+01:00,4,LAN
2026-01-23 07:00:00+01:00,26,LAN
2026-01-23 08:00:00+01:00,13,LAN
2026-01-23 09:00:00+01:00,16,LAN
2026-01-23 10:00:00+01:00,16,LAN
2026-01-23 11:00:00+01:00,22,LAN
2026-01-23 12:00:00+01:00,14,LAN


In [12]:
%%sql
-- Peak hours by time period (standard aviation periods)
SELECT 
    CASE 
        WHEN EXTRACT(HOUR FROM arrival_time__scheduled_utc) BETWEEN 6 AND 11 THEN 'Morning (06:00-11:59)'
        WHEN EXTRACT(HOUR FROM arrival_time__scheduled_utc) BETWEEN 12 AND 16 THEN 'Midday/Afternoon (12:00-16:59)'
        WHEN EXTRACT(HOUR FROM arrival_time__scheduled_utc) BETWEEN 17 AND 21 THEN 'Evening (17:00-21:59)'
        ELSE 'Night/Red-eye (22:00-05:59)'
    END AS time_period,
    COUNT(*) AS flight_count
FROM flights.flights_arrivals_raw
WHERE location_and_status__flight_leg_status != 'DEL'
GROUP BY time_period
ORDER BY flight_count DESC;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

time_period,flight_count
Midday/Afternoon (12:00-16:59),456
Evening (17:00-21:59),431
Morning (06:00-11:59),335
Night/Red-eye (22:00-05:59),200


### 2. Punctuality → status, scheduled vs actual times

In [13]:
%%sql
-- 2. Punctuality: Delay analysis for landed flights
SELECT 
    flight_id,
    arrival_time__scheduled_utc,
    arrival_time__actual_utc,
    EXTRACT(EPOCH FROM (arrival_time__actual_utc - arrival_time__scheduled_utc))/60 AS delay_minutes
FROM flights.flights_arrivals_raw
WHERE location_and_status__flight_leg_status = 'LAN'
  AND arrival_time__actual_utc IS NOT NULL
LIMIT 10;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

flight_id,arrival_time__scheduled_utc,arrival_time__actual_utc,delay_minutes
FR7619,2026-01-25 00:30:00+01:00,2026-01-25 00:13:31+01:00,-16.483333333333334
D84270,2026-01-25 00:05:00+01:00,2026-01-25 00:15:18+01:00,10.3
SK7674,2026-01-25 00:10:00+01:00,2026-01-24 23:59:25+01:00,-10.583333333333334
AY825,2026-01-25 00:20:00+01:00,2026-01-25 00:07:53+01:00,-12.116666666666667
SK1428,2026-01-25 00:25:00+01:00,2026-01-25 00:18:49+01:00,-6.183333333333334
NH221,2026-01-25 06:20:00+01:00,2026-01-25 06:31:03+01:00,11.05
SK701,2026-01-25 07:10:00+01:00,2026-01-25 06:55:18+01:00,-14.7
SK132,2026-01-25 07:55:00+01:00,2026-01-25 07:40:24+01:00,-14.6
LO453,2026-01-25 09:55:00+01:00,2026-01-25 09:41:59+01:00,-13.016666666666667
SK142,2026-01-25 09:25:00+01:00,2026-01-25 09:18:30+01:00,-6.5


### 3. Airline performance → Which airlines fly most? Delays by airline?

In [14]:
%%sql
-- 3a. Which airlines have the most arrivals?
SELECT 
    airline_operator__name AS airline,
    airline_operator__iata AS iata_code,
    COUNT(*) AS flight_count
FROM flights.flights_arrivals_raw
WHERE location_and_status__flight_leg_status != 'DEL'
GROUP BY airline_operator__name, airline_operator__iata
ORDER BY flight_count DESC
LIMIT 30;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

airline,iata_code,flight_count
SAS Scandinavian Airlines,SK,635
Norwegian,D8,141
Ryanair Ltd,FR,88
Lufthansa,LH,59
Finnair,AY,59
KLM,KL,44
Swiss,LX,36
PopulAir,HP,26
Air France,AF,25
British Airways,BA,24


In [15]:
%%sql
-- 3b. Average delay by airline (only landed flights with actual times)
SELECT 
    airline_operator__name AS airline,
    COUNT(*) AS landed_flights,
    ROUND(AVG(EXTRACT(EPOCH FROM (arrival_time__actual_utc - arrival_time__scheduled_utc))/60), 2) AS avg_delay_minutes,
    ROUND(MIN(EXTRACT(EPOCH FROM (arrival_time__actual_utc - arrival_time__scheduled_utc))/60), 2) AS best_early_minutes,
    ROUND(MAX(EXTRACT(EPOCH FROM (arrival_time__actual_utc - arrival_time__scheduled_utc))/60), 2) AS worst_late_minutes
FROM flights.flights_arrivals_raw
WHERE location_and_status__flight_leg_status = 'LAN'
  AND arrival_time__actual_utc IS NOT NULL
GROUP BY airline_operator__name
HAVING COUNT(*) >= 3
ORDER BY avg_delay_minutes DESC;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

airline,landed_flights,avg_delay_minutes,best_early_minutes,worst_late_minutes
TAP Portugal,3,16.73,14.67,20.85
Sunclass Airlines,14,15.26,-2.52,39.05
Tui Fly Nordic,7,14.14,-33.72,93.0
Pegasus Airlines,14,12.31,-29.15,107.32
Jettime,4,9.86,-6.0,21.0
Norse Atlantic Airways,5,7.65,-20.07,14.88
AJet,5,7.26,3.5,14.5
Austrian Airlines,7,6.19,-17.35,58.87
Norwegian,117,4.18,-33.28,100.32
Icelandair,5,3.79,1.17,6.08


### 4. Route popularity → airport pairs

In [16]:
%%sql
-- 4. Most popular routes (airport pairs)
SELECT 
    flight_leg_identifier__departure_airport_iata AS origin,
    flight_leg_identifier__arrival_airport_iata AS destination,
    departure_airport_english AS origin_name,
    COUNT(*) AS flight_count
FROM flights.flights_arrivals_raw
WHERE location_and_status__flight_leg_status != 'DEL'
GROUP BY 
    flight_leg_identifier__departure_airport_iata, 
    flight_leg_identifier__arrival_airport_iata,
    departure_airport_english
ORDER BY flight_count DESC
LIMIT 20;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

origin,destination,origin_name,flight_count
CPH,ARN,Copenhagen,61
HEL,ARN,Helsinki,55
ARN,LLA,Stockholm ARN,51
LLA,ARN,Luleå,45
OSL,ARN,Oslo,42
ARN,UME,Stockholm ARN,35
AMS,ARN,Amsterdam,34
LHR,ARN,London LHR,33
UME,ARN,Umeå,32
CDG,ARN,Paris CDG,25


In [17]:
%%sql
-- 4. Most popular routes with full airport names
WITH swedish_airports AS (
    SELECT 'ARN' AS iata, 'Stockholm Arlanda Airport' AS name UNION ALL
    SELECT 'BMA', 'Bromma Stockholm Airport' UNION ALL
    SELECT 'GOT', 'Göteborg Landvetter Airport' UNION ALL
    SELECT 'MMX', 'Malmö Airport' UNION ALL
    SELECT 'LLA', 'Luleå Airport' UNION ALL
    SELECT 'UME', 'Umeå Airport' UNION ALL
    SELECT 'OSD', 'Åre Östersund Airport' UNION ALL
    SELECT 'VBY', 'Visby Airport' UNION ALL
    SELECT 'RNB', 'Ronneby Airport' UNION ALL
    SELECT 'KRN', 'Kiruna Airport'
)
SELECT 
    f.flight_leg_identifier__departure_airport_iata AS origin_code,
    f.departure_airport_english AS origin_name,
    f.flight_leg_identifier__arrival_airport_iata AS destination_code,
    sa.name AS destination_name,
    COUNT(*) AS flight_count
FROM flights.flights_arrivals_raw f
LEFT JOIN swedish_airports sa 
    ON f.flight_leg_identifier__arrival_airport_iata = sa.iata
WHERE f.location_and_status__flight_leg_status != 'DEL'
GROUP BY 
    f.flight_leg_identifier__departure_airport_iata,
    f.departure_airport_english,
    f.flight_leg_identifier__arrival_airport_iata,
    sa.name
ORDER BY flight_count DESC
LIMIT 20;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

origin_code,origin_name,destination_code,destination_name,flight_count
CPH,Copenhagen,ARN,Stockholm Arlanda Airport,61
HEL,Helsinki,ARN,Stockholm Arlanda Airport,55
ARN,Stockholm ARN,LLA,Luleå Airport,51
LLA,Luleå,ARN,Stockholm Arlanda Airport,45
OSL,Oslo,ARN,Stockholm Arlanda Airport,42
ARN,Stockholm ARN,UME,Umeå Airport,35
AMS,Amsterdam,ARN,Stockholm Arlanda Airport,34
LHR,London LHR,ARN,Stockholm Arlanda Airport,33
UME,Umeå,ARN,Stockholm Arlanda Airport,32
GOT,Göteborg,ARN,Stockholm Arlanda Airport,25


In [18]:
%%sql
-- 4. Most popular routes with domestic/international classification
WITH swedish_airports AS (
    SELECT 'ARN' AS iata, 'Stockholm Arlanda Airport' AS name UNION ALL
    SELECT 'BMA', 'Bromma Stockholm Airport' UNION ALL
    SELECT 'GOT', 'Göteborg Landvetter Airport' UNION ALL
    SELECT 'MMX', 'Malmö Airport' UNION ALL
    SELECT 'LLA', 'Luleå Airport' UNION ALL
    SELECT 'UME', 'Umeå Airport' UNION ALL
    SELECT 'OSD', 'Åre Östersund Airport' UNION ALL
    SELECT 'VBY', 'Visby Airport' UNION ALL
    SELECT 'RNB', 'Ronneby Airport' UNION ALL
    SELECT 'KRN', 'Kiruna Airport'
)
SELECT 
    f.flight_leg_identifier__departure_airport_iata AS origin_code,
    f.departure_airport_english AS origin_name,
    f.flight_leg_identifier__arrival_airport_iata AS destination_code,
    sa_dest.name AS destination_name,
    CASE 
        WHEN sa_origin.iata IS NOT NULL THEN 'Domestic'
        ELSE 'International'
    END AS route_type,
    COUNT(*) AS flight_count
FROM flights.flights_arrivals_raw f
LEFT JOIN swedish_airports sa_dest 
    ON f.flight_leg_identifier__arrival_airport_iata = sa_dest.iata
LEFT JOIN swedish_airports sa_origin
    ON f.flight_leg_identifier__departure_airport_iata = sa_origin.iata
WHERE f.location_and_status__flight_leg_status != 'DEL'
GROUP BY 
    f.flight_leg_identifier__departure_airport_iata,
    f.departure_airport_english,
    f.flight_leg_identifier__arrival_airport_iata,
    sa_dest.name,
    route_type
ORDER BY flight_count DESC
LIMIT 20;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

origin_code,origin_name,destination_code,destination_name,route_type,flight_count
CPH,Copenhagen,ARN,Stockholm Arlanda Airport,International,61
HEL,Helsinki,ARN,Stockholm Arlanda Airport,International,55
ARN,Stockholm ARN,LLA,Luleå Airport,Domestic,51
LLA,Luleå,ARN,Stockholm Arlanda Airport,Domestic,45
OSL,Oslo,ARN,Stockholm Arlanda Airport,International,42
ARN,Stockholm ARN,UME,Umeå Airport,Domestic,35
AMS,Amsterdam,ARN,Stockholm Arlanda Airport,International,34
LHR,London LHR,ARN,Stockholm Arlanda Airport,International,33
UME,Umeå,ARN,Stockholm Arlanda Airport,Domestic,32
GOT,Göteborg,ARN,Stockholm Arlanda Airport,Domestic,25


In [19]:
%%sql
-- 4b. Domestic vs International flight distribution
WITH swedish_airports AS (
    SELECT 'ARN' AS iata, 'Stockholm Arlanda Airport' AS name UNION ALL
    SELECT 'BMA', 'Bromma Stockholm Airport' UNION ALL
    SELECT 'GOT', 'Göteborg Landvetter Airport' UNION ALL
    SELECT 'MMX', 'Malmö Airport' UNION ALL
    SELECT 'LLA', 'Luleå Airport' UNION ALL
    SELECT 'UME', 'Umeå Airport' UNION ALL
    SELECT 'OSD', 'Åre Östersund Airport' UNION ALL
    SELECT 'VBY', 'Visby Airport' UNION ALL
    SELECT 'RNB', 'Ronneby Airport' UNION ALL
    SELECT 'KRN', 'Kiruna Airport'
)
SELECT 
    CASE 
        WHEN sa_origin.iata IS NOT NULL THEN 'Domestic'
        ELSE 'International'
    END AS route_type,
    COUNT(*) AS total_flights,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM flights.flights_arrivals_raw f
LEFT JOIN swedish_airports sa_origin
    ON f.flight_leg_identifier__departure_airport_iata = sa_origin.iata
WHERE f.location_and_status__flight_leg_status != 'DEL'
GROUP BY route_type
ORDER BY total_flights DESC;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

route_type,total_flights,percentage
International,1036,72.86
Domestic,386,27.14


### 5. Capacity utilization → flight counts per airport

In [20]:
%%sql
-- 5. Capacity utilization: Flight counts per Swedish airport
WITH swedish_airports AS (
    SELECT 'ARN' AS iata, 'Stockholm Arlanda Airport' AS name UNION ALL
    SELECT 'BMA', 'Bromma Stockholm Airport' UNION ALL
    SELECT 'GOT', 'Göteborg Landvetter Airport' UNION ALL
    SELECT 'MMX', 'Malmö Airport' UNION ALL
    SELECT 'LLA', 'Luleå Airport' UNION ALL
    SELECT 'UME', 'Umeå Airport' UNION ALL
    SELECT 'OSD', 'Åre Östersund Airport' UNION ALL
    SELECT 'VBY', 'Visby Airport' UNION ALL
    SELECT 'RNB', 'Ronneby Airport' UNION ALL
    SELECT 'KRN', 'Kiruna Airport'
)
SELECT 
    sa.iata AS airport_code,
    sa.name AS airport_name,
    COUNT(*) AS total_arrivals,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS percentage_of_traffic
FROM flights.flights_arrivals_raw f
INNER JOIN swedish_airports sa 
    ON f.flight_leg_identifier__arrival_airport_iata = sa.iata
WHERE f.location_and_status__flight_leg_status != 'DEL'
GROUP BY sa.iata, sa.name
ORDER BY total_arrivals DESC;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

airport_code,airport_name,total_arrivals,percentage_of_traffic
ARN,Stockholm Arlanda Airport,965,67.86
GOT,Göteborg Landvetter Airport,235,16.53
LLA,Luleå Airport,63,4.43
MMX,Malmö Airport,43,3.02
UME,Umeå Airport,38,2.67
VBY,Visby Airport,26,1.83
OSD,Åre Östersund Airport,17,1.2
KRN,Kiruna Airport,16,1.13
BMA,Bromma Stockholm Airport,11,0.77
RNB,Ronneby Airport,8,0.56


In [21]:
%%sql
-- 5b. Complete capacity utilization: Arrivals + Departures per airport
WITH swedish_airports AS (
    SELECT 'ARN' AS iata, 'Stockholm Arlanda Airport' AS name UNION ALL
    SELECT 'BMA', 'Bromma Stockholm Airport' UNION ALL
    SELECT 'GOT', 'Göteborg Landvetter Airport' UNION ALL
    SELECT 'MMX', 'Malmö Airport' UNION ALL
    SELECT 'LLA', 'Luleå Airport' UNION ALL
    SELECT 'UME', 'Umeå Airport' UNION ALL
    SELECT 'OSD', 'Åre Östersund Airport' UNION ALL
    SELECT 'VBY', 'Visby Airport' UNION ALL
    SELECT 'RNB', 'Ronneby Airport' UNION ALL
    SELECT 'KRN', 'Kiruna Airport'
),
arrivals AS (
    SELECT 
        flight_leg_identifier__arrival_airport_iata AS airport_code,
        COUNT(*) AS arrival_count
    FROM flights.flights_arrivals_raw
    WHERE location_and_status__flight_leg_status != 'DEL'
    GROUP BY flight_leg_identifier__arrival_airport_iata
),
departures AS (
    SELECT 
        flight_leg_identifier__departure_airport_iata AS airport_code,
        COUNT(*) AS departure_count
    FROM flights.flights_departures_raw
    WHERE location_and_status__flight_leg_status != 'DEL'
    GROUP BY flight_leg_identifier__departure_airport_iata
)
SELECT 
    sa.iata AS airport_code,
    sa.name AS airport_name,
    COALESCE(a.arrival_count, 0) AS arrivals,
    COALESCE(d.departure_count, 0) AS departures,
    COALESCE(a.arrival_count, 0) + COALESCE(d.departure_count, 0) AS total_movements,
    ROUND((COALESCE(a.arrival_count, 0) + COALESCE(d.departure_count, 0)) * 100.0 / 
          SUM(COALESCE(a.arrival_count, 0) + COALESCE(d.departure_count, 0)) OVER (), 2) AS percentage_of_total
FROM swedish_airports sa
LEFT JOIN arrivals a ON sa.iata = a.airport_code
LEFT JOIN departures d ON sa.iata = d.airport_code
ORDER BY total_movements DESC;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

airport_code,airport_name,arrivals,departures,total_movements,percentage_of_total
ARN,Stockholm Arlanda Airport,965,976,1941,68.27
GOT,Göteborg Landvetter Airport,235,235,470,16.53
LLA,Luleå Airport,63,57,120,4.22
MMX,Malmö Airport,43,41,84,2.95
UME,Umeå Airport,38,36,74,2.6
VBY,Visby Airport,26,26,52,1.83
OSD,Åre Östersund Airport,17,16,33,1.16
KRN,Kiruna Airport,16,16,32,1.13
BMA,Bromma Stockholm Airport,11,11,22,0.77
RNB,Ronneby Airport,8,7,15,0.53


### 6. Seasonal trends → dates, day of week

In [22]:
%%sql
-- 6. Seasonal trends: Flight distribution by day of week
SELECT 
    STRFTIME(arrival_time__scheduled_utc, '%A') AS day_of_week,
    EXTRACT(ISODOW FROM arrival_time__scheduled_utc) AS day_number,
    COUNT(*) AS flight_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM flights.flights_arrivals_raw
WHERE location_and_status__flight_leg_status != 'DEL'
GROUP BY 
    STRFTIME(arrival_time__scheduled_utc, '%A'),
    EXTRACT(ISODOW FROM arrival_time__scheduled_utc)
ORDER BY day_number;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

day_of_week,day_number,flight_count,percentage
Friday,5,344,24.19
Saturday,6,422,29.68
Sunday,7,656,46.13


## 6. Data Quality Checks

In [23]:
%%sql
-- Check for NULL values in key fields (arrivals)
SELECT 
    COUNT(*) as total_rows,
    COUNT(flight_id) as has_flight_id,
    COUNT(arrival_time__scheduled_utc) as has_scheduled,
    COUNT(arrival_time__actual_utc) as has_actual,
    COUNT(location_and_status__flight_leg_status) as has_status,
    COUNT(airline_operator__name) as has_airline,
    COUNT(flight_leg_identifier__departure_airport_iata) as has_origin,
    COUNT(flight_leg_identifier__arrival_airport_iata) as has_destination
FROM flights.flights_arrivals_raw;

Running query in 'duckdb:///../data_warehouse/svenska-flyt.duckdb'

total_rows,has_flight_id,has_scheduled,has_actual,has_status,has_airline,has_origin,has_destination
1801,1801,1801,1221,1801,1801,1801,1801


## 7. EDA Findings & Recommendations

### Full Column List and Meanings

**Key Columns (All in flights.flights_arrivals_raw):**
- `flight_id`: Unique flight identifier (e.g., "FI312")
- `airline_operator__name`, `airline_operator__iata`: Airline info
- `flight_leg_identifier__departure_airport_iata`: Origin airport code
- `flight_leg_identifier__arrival_airport_iata`: Destination airport code
- `departure_airport_english`: Readable origin airport name
- `arrival_time__scheduled_utc`: Scheduled arrival time (UTC+1)
- `arrival_time__estimated_utc`: Estimated arrival (if updated)
- `arrival_time__actual_utc`: Actual arrival time (when landed)
- `location_and_status__flight_leg_status`: Flight status (SCH, LAN, DEL, CAN, etc.)
- `location_and_status__terminal`, `location_and_status__gate`: Airport location
- `baggage__first_bag_utc`, `baggage__last_bag_utc`: Baggage handling times

### Data Types & Conversions Needed

| Column | Type | Conversion Needed |
|--------|------|------------------|
| flight_id | VARCHAR | Keep as string |
| *_utc timestamps | TIMESTAMP WITH TIME ZONE | Convert to local (CET/CEST) for reporting |
| IATA codes | VARCHAR | Standardize to uppercase |
| Status codes | VARCHAR | Map to meaningful labels (SCH→"Scheduled", LAN→"Landed") |

### Null Handling Strategy

| Column | Nulls | Strategy |
|--------|-------|----------|
| arrival_time__actual_utc | 541/798 (68%) | Expected for non-landed flights; filter WHERE actual_utc IS NOT NULL for delay analysis |
| arrival_time__estimated_utc | All NULL | Not used by Swedavia API; ignore |
| location_and_status__gate | Most NULL | Expected until flight arrives; not critical for KPIs |
| flight_leg_identifier__* | 0 | All complete; use for routing analysis |

### Key Fields by KPI

1. **Peak Hours**: `arrival_time__scheduled_utc` + `EXTRACT(HOUR)` + time period CASE statement
2. **Punctuality**: `arrival_time__actual_utc` vs `arrival_time__scheduled_utc` (only LAN status)
3. **Airline Performance**: `airline_operator__name` + delay calculations (avg, min, max)
4. **Route Popularity**: `flight_leg_identifier__departure_airport_iata` + `flight_leg_identifier__arrival_airport_iata` + Swedish airport lookup table
5. **Capacity Utilization**: Airport IATA codes aggregated by arrival/departure counts
6. **Seasonal Trends**: `STRFTIME(arrival_time__scheduled_utc, '%A')` for day-of-week patterns

### dbt Staging Model Design

**Initial `stg_flights_arrivals` model:**
- **Keep ALL flights including DEL (deleted) flights** — track at source, don't filter
- Add deletion tracking flags for downstream analysis:
  - `is_deleted = flight_status = 'DEL'` (boolean, 142/798 = 18%)
  - `is_cancelled = flight_status = 'CAN'` (boolean, 2/798)
- Create `route_key` compound identifier:
  - `route_key = origin_airport_iata || '-' || destination_airport_iata` (e.g., 'ARN-GOT')
  - Use for deletion_ratio analysis: COUNT(is_deleted) / COUNT(*) per route
- Rename flattened columns for readability:
  - `flight_leg_identifier__departure_airport_iata` → `origin_airport_iata`
  - `arrival_time__scheduled_utc` → `scheduled_arrival_utc`
  - `location_and_status__flight_leg_status` → `flight_status`
- Create calculated fields:
  - `delay_minutes = EXTRACT(EPOCH FROM (actual_arrival_utc - scheduled_arrival_utc))/60` (only for LAN flights)
  - `arrival_hour = EXTRACT(HOUR FROM scheduled_arrival_utc)`
  - `day_of_week = STRFTIME(scheduled_arrival_utc, '%A')`
- Add metadata:
  - `data_load_date` (from `_dlt_load_id` timestamp)
  - `is_landed = flight_status = 'LAN'`
  - `is_domestic = origin_airport_iata IN ('ARN', 'BMA', 'GOT', 'MMX', 'LLA', 'UME', 'OSD', 'VBY', 'RNB', 'KRN')`

**Downstream Model: Route Cancellation Analysis**
- Create `route_cancellation_stats` model from `stg_flights_arrivals`
- GROUP BY `route_key`, aggregate deletion ratios
- Ready for weather data enrichment: JOIN with weather on `date` + `route_key` to identify weather-correlated cancellation patterns

**Key Insight**: Only 257/798 flights (32%) have actual times — need 7-day backfill for robust punctuality analysis. Deletion tracking enables weather correlation research without losing data at source.